<a href="https://colab.research.google.com/github/JAYKIM96/APIfromjay/blob/main/%EB%B8%8C%EB%9E%9C%EB%93%9CCIC%EA%B5%90%EC%9C%A1%EC%9A%A9_%EC%9C%A0%ED%8A%9C%EB%B8%8CAPI(%EC%9B%90%EB%B3%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google colab이란?
- 웹 브라우저에서 Python 코드를 작성하고 실행할 수 있는 환경


## API란?
- 일종의 MENU : 개발자들은 API를 통해 특정 기능이나 데이터에 접근할 수 있음
      ex. 날씨 정보를 제공하는 API : 개발자는 API를 통해 날씨 정보를 얻고 자신의 어플리케이션에 표시할 수 있음

###코드 순서
1. 검색어에 따른 유튜브 데이터 추출하기
2. 특정 유튜브 콘텐츠에 대한 모든 댓글 추출하기

###주의할 점
- API키는 외부에 게시하거나 공유하는 등의 행위를 자제해주세요.
  - 데이터를 요청할 때마다 사용 가능한 할당량이 깎여 나가는 구조이기 때문



###기간 1개, 키워드 1개만 조회하고 싶을 땐 어떻게 하나요?
- 해당 부분을 아래와 같이 수정해보세요.

```
def main():
    search_dates = [
        ["2022-08-01T00:00:00Z", "2023-07-31T00:00:00Z"]
    ]

    keywords = ["등드름"]  
    excel_file_name = '내맘대로파일명.xlsx'  
```

#검색어에 따른 유튜브 데이터 추출하기

In [ ]:
!pip install xlsxwriter
from google.colab import files  # Google Colab 파일 다운로드 기능을 위한 모듈 추가
from googleapiclient.discovery import build
import pandas as pd
import datetime as dt

class YoutubeVideoapi:
    def __init__(self, date_ranges, keyword):
        self.developer_key = 'API KEY' #현대홈쇼핑 API키 발급받은 코드를 기입 (단, 유출에 주의)
        self.youtube_api_service_name = "youtube"
        self.youtube_api_version = 'v3'
        self.date_ranges = date_ranges
        self.keyword = keyword

    def videolist(self):
        youtube = build(self.youtube_api_service_name, self.youtube_api_version, developerKey=self.developer_key)
        data_dicts = {
            '검색날짜': [], '키워드': [], '크리에이터명': [], '제목': [],
            '조회수': [], '댓글수': [], '좋아요수': [], '게시일': [],
            '구독자수': [], '영상 설명': [], '상위 댓글': [], 'URL': []
        }


        # 조회수 기준 상위 5~60개 영상을 집계해달라 (수정하고 싶을 경우, 60에 해당하는 숫자를 변경)
        for date_range in self.date_ranges:
            nextPageToken = None
            count = 0  # 각 기간별로 카운트하기 위한 변수

            while count < 60:
                search_response = youtube.search().list(
                    q=self.keyword,
                    order='viewCount',
                    part='snippet',
                    maxResults=60 - count,
                    publishedAfter=date_range[0],
                    publishedBefore=date_range[1],
                    pageToken=nextPageToken
                ).execute()

                for item in search_response['items']:
                    if count >= 60:
                        break

                    channel_id = item['snippet']['channelId']
                    video_id = item['id']['videoId']

                    video_info = youtube.videos().list(
                        part='snippet, statistics',
                        id=video_id,
                    ).execute()

                    channel_info = youtube.channels().list(
                        part="statistics,snippet",
                        id=channel_id
                    ).execute()


                    #고정댓글을 불러오기 위한 대안으로, 최상단 위치한 댓글 불러오기 (비활성화인 경우 비활성화로 표기)
                    top_comment = "댓글 비활성화"
                    try:
                        comments_response = youtube.commentThreads().list(
                            part='snippet',
                            videoId=video_id,
                            maxResults=1,
                            order='relevance'
                        ).execute()

                        if comments_response['items']:
                            top_comment = comments_response['items'][0]['snippet']['topLevelComment']['snippet']['textDisplay']
                    except:
                        pass

                    # 비디오의 URL을 생성
                    video_url = f"https://www.youtube.com/watch?v={video_id}"


                    # 집계하고자 하는 데이터
                    data_dicts['검색날짜'].append(str(dt.date.today()))
                    data_dicts['키워드'].append(self.keyword)
                    data_dicts['크리에이터명'].append(channel_info['items'][0]['snippet'].get('title'))
                    data_dicts['제목'].append(video_info['items'][0]['snippet'].get('title'))
                    data_dicts['조회수'].append(video_info['items'][0]['statistics'].get('viewCount', "0"))
                    data_dicts['댓글수'].append(video_info['items'][0]['statistics'].get('commentCount', "0"))
                    data_dicts['좋아요수'].append(video_info['items'][0]['statistics'].get('likeCount', "0"))
                    data_dicts['게시일'].append(str(video_info['items'][0]['snippet'].get('publishedAt')))
                    data_dicts['구독자수'].append(channel_info['items'][0]['statistics'].get('subscriberCount', "0"))
                    data_dicts['영상 설명'].append(video_info['items'][0]['snippet'].get('description'))
                    data_dicts['상위 댓글'].append(top_comment)
                    data_dicts['URL'].append(video_url)

                    count += 1

                nextPageToken = search_response.get('nextPageToken')

                if not nextPageToken:
                    break

        return data_dicts

#언제부터 언제까지의 기간에 걸친 데이터가 필요한가? (각각의 기간 내 게시된 키워드에 대한 콘텐츠 조회)

def main():
    search_dates = [
        ["2021-08-01T00:00:00Z", "2022-07-31T00:00:00Z"],
        ["2022-08-01T00:00:00Z", "2023-07-31T00:00:00Z"]
    ]

    keywords = ["키워드1","키워드2","키워드3"]  # 검색하기 원하는 키워드명(""와 ,로 추가)
    excel_file_name = '원하는 파일명.xlsx'  # 저장하기 원하는 파일명 (수정 가능하며 '작은따옴표필수')

    with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:
        for keyword in keywords:
            search_dict = YoutubeVideoapi(date_ranges=search_dates, keyword=keyword).videolist()
            df = pd.DataFrame(search_dict)
            df.to_excel(writer, sheet_name=keyword, index=False, encoding='utf-8')

    print(f"Results saved to {excel_file_name}")

    # Google Colab에서 파일 다운로드
    files.download(excel_file_name)

main()

#마지막으로는 Ctrl+Enter를 쳐주시면 됩니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 3.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


Results saved to 예시.xlsx


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#특정 유튜브 콘텐츠에 대한 모든 댓글 추출하기

※참고 사항


```
# response = api_obj.commentThreads().list(part='snippet', videoId='8k5pmcHfwRA', maxResults=100).execute()
```


- 이 코드에서 VideoId : **콘텐츠 URL에 써있는 & 앞까지의 문자 복붙**

ex.https://www.youtube.com/watch?v=8k5pmcHfwRA&t=389s 이면, '8k5pmcHfwRA'
- 작은 따옴표 필수!!


In [ ]:
!pip install google-api-python-client
import pandas as pd
from googleapiclient.discovery import build
from google.colab import files

import warnings  # 경고창 무시
warnings.filterwarnings('ignore')

comments = list()
api_obj = build('youtube', 'v3', developerKey='API KEY')  # 현대홈쇼핑 API키 발급받은 코드를 기입 (단, 유출에 주의)
response = api_obj.commentThreads().list(part='snippet',
videoId='8k5pmcHfwRA', maxResults=100).execute() # VideoId에 콘텐츠 URL에 써있는 & 앞까지의 문자 복붙

while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet', videoId='8k5pmcHfwRA', pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

df = pd.DataFrame(comments, columns=['Comment', 'Author', 'Published At', 'Like Count'])
filename = "원하는 파일명.xlsx" # 저장하기 원하는 파일명 (수정 가능하며 "작은따옴표필수")
df.to_excel(filename, index=False)

# Google Colab에서 파일 다운로드 기능
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>